In [1]:
import pandas as pd
import numpy as np

In [2]:
# LOAD FILE INTO PROGRAM " 'name' = 'LOC' "
file_to_load = "../Resources/purchase_data.csv"

## Load and Read File

In [3]:
# READ CSV FILE 
purchase_data_df = pd.read_csv(file_to_load)

#PRINT TABLE
purchase_data_df.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


In [4]:
# COUNT HOW MANY PLAYERS THERE ARE IN THE DATA
total_players = purchase_data_df.groupby(['SN'])
total_players = total_players.count()
number_rows = total_players.shape[0]

print('Number of Rows in DataFrame :',number_rows)

Number of Rows in DataFrame : 576


In [5]:
# CREATE NEW TABLE while including data from 'purchase_data_df.count()'
list_players = pd.DataFrame({
    "Total of Players":[number_rows]
})

#PRINT TABLE
list_players.head()

,Total of Players
0,576


In [6]:
# FINDING THE NUMBER OF UNIQUE ITEMS

items = purchase_data_df['Item Name'].value_counts()
# Prints out full raw data (shows all rows) 
# np.savetxt('LOCATION TO SAVE plus name', 'Data Frame', FMT) # fmt = '%d' rounds up the values to integers
np.savetxt('../Resources/unique_items_data.text', items, fmt='%d')
number_of_unique_items = items.shape[0]
print("Number of Unique Items: " + str(number_of_unique_items) )

Number of Unique Items: 179


## Finding the average price

In [7]:
# Print out the amount of purchases that happened in the given data
                # count how many there are... 780 was the outcome of all columns
amount_purchased = purchase_data_df.count()
# did not matter which column to choose...but, chose 'Purchase ID' because it will tell us how many items are purchased
amount_purchased["Purchase ID"]

780

In [8]:
# FINDING THE AVERAGE PRICE

price_df = purchase_data_df['Price']
total_revenue = price_df.sum()                       # TOTAL REVENUE by using '.sum()'
price_average = ((total_revenue)/(amount_purchased["Purchase ID"])) # COULD ADD 'round(decimals = 2)' to round it up, 
                                                     # but we use '.astype(float).map("${:,.2f}".format)' instead

print("Price Average: " + str(price_average))
print("")
print("Total Rev.: " + str(total_revenue))

Price Average: 3.0509871794871795

Total Rev.: 2379.77


In [9]:
# CREATE TABLE
purchasing_analysis_df = pd.DataFrame({
    "Number of Unique Items": [number_of_unique_items],
    "Average Price": [price_average],
    "Number of Purchases": [amount_purchased["Purchase ID"]],
    "Total Revenue":[total_revenue]
})

#TO ROUND VALUES UP AND ADD '$'
purchasing_analysis_df["Average Price"] = purchasing_analysis_df["Average Price"].astype(float).map(
    "${:,.2f}".format)
purchasing_analysis_df["Total Revenue"] = purchasing_analysis_df["Total Revenue"].astype(float).map(
    "${:,.2f}".format)


# PRINT OUT TABLE
purchasing_analysis_df

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,179,$3.05,780,"$2,379.77"


## Gender Table's

In [10]:
# Gender Count in column of 'Gender' only
gender_count = purchase_data_df["Gender"].value_counts()
gender_count.head()

# CREATE TABLE COUNT of Genders
                                        #groupby forces 'Gender' to become the index
grouped_gender_count = purchase_data_df.groupby(['Gender'])

# The object returned is a "GroupBy" object and cannot be viewed normally...
print(grouped_gender_count)

# Data Frame broke into count only ('Total Count' = [Male, Female, Others])
gender_count_test = grouped_gender_count.count().head()

# delete the extra columns (NOT NEEDED)
del gender_count_test['Price']
del gender_count_test['SN']
del gender_count_test['Item ID']
del gender_count_test['Item Name']
del gender_count_test['Age']

# Keep Purchase ID, but convert 'Purchase ID' into 'Total Count'
gender_count_test = gender_count_test.rename(columns={"Purchase ID": "Total Count"})

# to find the amount of genders 
gender_total = gender_count_test.sum()

# the amount of males/females/others... found in 'gender_count_test' data frame... and divide it by the amount of genders
female_amount = gender_count_test["Total Count"]["Female"] 
male_amount = gender_count_test["Total Count"]["Male"] 
other_amount = gender_count_test["Total Count"]["Other / Non-Disclosed"] 


male_percentage = (male_amount/gender_total)
female_percentage = (female_amount/gender_total)
other_percentage = (other_amount/gender_total)
# What the code will print out... as gender_total is 780
    # male_percentage = (652/gender_total)
    # female_percentage = (113/gender_total)
    # other_percentage = (15/gender_total)

# added a column with its values into 'gender_count_test' data frame
gender_count_test.loc[:, 'PercentageofPlayers'] = [male_percentage, female_percentage, other_percentage]

# convert the 'PercentageofPlayers' values into percetange and string ( example:  .8346121 to 83.46121  )
gender_count_test.PercentageofPlayers = (gender_count_test.PercentageofPlayers * 100).astype(str)

#TO ROUND VALUES UP AND ADD '%'
gender_count_test["Percentage of Players"] = gender_count_test["PercentageofPlayers"].astype(float).map("{:,.2f}%".format)

# DEL the extra COLUMN
del gender_count_test['PercentageofPlayers']

# print out table
gender_count_test.head()

,Total Count,Percentage of Players
Gender,,
Female,113,83.59%
Male,652,14.49%
Other / Non-Disclosed,15,1.92%


In [11]:

# MALE ONLY
male_gender_only = purchase_data_df.loc[purchase_data_df["Gender"] == "Male", :]
male_gender_only.head()

male_avg_price = male_gender_only["Price"].mean()
male_purchase_total = male_gender_only["Price"].sum()


# FEMALE ONLY
female_gender_only = purchase_data_df.loc[purchase_data_df["Gender"] == "Female", :]
female_gender_only.head()

female_avg_price = female_gender_only["Price"].mean()
female_purchase_total = female_gender_only["Price"].sum()


# OTHER ONLY
other_gender_only = purchase_data_df.loc[purchase_data_df["Gender"] == "Other / Non-Disclosed", :]
other_gender_only.head()

other_avg_price = other_gender_only["Price"].mean()
other_purchase_total = other_gender_only["Price"].sum()

In [12]:
# MALE ONLY 
grouped_male_gender_df = male_gender_only.groupby(['SN'])

    # Each Person who purchased more than one will add their prices... example Aeda94: 'Price' = [2.00, 3.00, 1.00]
    #                                                                          Yasur85: 'Price' = [1.00, .10]
    #                                                                  Outcome of code: Aeda 94: 6.00   Yasur85: 1.10
grp_male_sum = grouped_male_gender_df["Price"].sum() 

    # After group_by and sum of each 'SN', apply .mean() to caculate by adding each 'SN' then divide by the amount of 'SN'
    # example: 'Lism78' = 5.10, 'Lisovynya38' = 3.90... then Lism78 + Lisovyna38 = 9.00 then 9.00/2 = the avg total purchase per person
grp_male_avg_per_person = grp_male_sum.mean()

# FEMALE ONLY
grouped_female_gender_df = female_gender_only.groupby(['SN'])
grp_female_sum = grouped_female_gender_df["Price"].sum() 
grp_female_avg_per_person = grp_female_sum.mean()


# OTHER ONLY
grouped_other_gender_df = other_gender_only.groupby(['SN'])
grp_other_sum = grouped_other_gender_df["Price"].sum() 
grp_other_avg_per_person = grp_other_sum.mean()

In [13]:
gender_analysis = grouped_gender_count.count().head()

gender_analysis_table_df = gender_analysis.drop(['SN', 'Age', 'Item ID', 'Item Name', 'Price'], axis=1)

gender_analysis_table_df = gender_analysis_table_df.rename(columns={"Purchase ID": "Purchase Count"})

gender_analysis_table_df.head()


# Add COLUMNS and VALUES into Table
gender_analysis_table_df.loc[:, 'Average Purchase of Price'] = [female_avg_price, male_avg_price, other_avg_price]
gender_analysis_table_df.loc[:, 'Total Purchase Value'] = [female_purchase_total, male_purchase_total, other_purchase_total]
gender_analysis_table_df.loc[:, 'Avg Total Purchase per Person'] = [grp_female_avg_per_person, grp_male_avg_per_person, grp_other_avg_per_person]


#TO ROUND VALUES UP AND ADD '$'
gender_analysis_table_df["Average Purchase of Price"] = gender_analysis_table_df["Average Purchase of Price"].astype(float).map("${:,.2f}".format)
gender_analysis_table_df["Total Purchase Value"] = gender_analysis_table_df["Total Purchase Value"].astype(float).map("${:,.2f}".format)
gender_analysis_table_df["Avg Total Purchase per Person"] = gender_analysis_table_df["Avg Total Purchase per Person"].astype(float).map("${:,.2f}".format)


gender_analysis_table_df.head()


,Purchase Count,Average Purchase of Price,Total Purchase Value,Avg Total Purchase per Person
Gender,,,,
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,"$1,967.64",$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


In [14]:
print(purchase_data_df["Age"].max())
print(purchase_data_df["Age"].min())

45
7


In [15]:
age_bins = purchase_data_df
# Create bins in which to place values based upon TED Talk views
bins = [0, 9, 14, 19, 24, 29, 34, 39, 50]
# Create the names for the five bins
group_names = ["<10", "10-14", "15-19", "20-24", "25-29","30-34","35-39","40+"]

age_bins[""] = pd.cut(age_bins["Age"], bins, labels=group_names, include_lowest=True)
age_bins

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price,
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53,20-24
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56,40+
2,2,Ithergue48,24,Male,92,Final Critic,4.88,20-24
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27,20-24
4,4,Iskosia90,23,Male,131,Fury,1.44,20-24
...,...,...,...,...,...,...,...,...
775,775,Aethedru70,21,Female,60,Wolf,3.54,20-24
776,776,Iral74,21,Male,164,Exiled Doomblade,1.63,20-24
777,777,Yathecal72,20,Male,67,"Celeste, Incarnation of the Corrupted",3.46,20-24
778,778,Sisur91,7,Male,92,Final Critic,4.19,<10


In [30]:
# Drops all 'SN' that have the same 'SN' since we do not want duplicates of 'Age' to change the data

age_bins = age_bins.drop_duplicates('SN',keep='last')

# How to check if it went through... check the amount of rows... we had '780' rows and by using '.drop_dplicates', 
# we end up with only '576' rows which is the amount of players. Hence, this is correct

# Print data frame
age_bins


,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price,
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56,40+
2,2,Ithergue48,24,Male,92,Final Critic,4.88,20-24
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27,20-24
4,4,Iskosia90,23,Male,131,Fury,1.44,20-24
6,6,Itheria73,36,Male,169,"Interrogator, Blood Blade of the Queen",2.18,35-39
...,...,...,...,...,...,...,...,...
775,775,Aethedru70,21,Female,60,Wolf,3.54,20-24
776,776,Iral74,21,Male,164,Exiled Doomblade,1.63,20-24
777,777,Yathecal72,20,Male,67,"Celeste, Incarnation of the Corrupted",3.46,20-24
778,778,Sisur91,7,Male,92,Final Critic,4.19,<10


In [107]:
# Create a GroupBy object based upon "View Group"
age_group = age_bins.groupby("")

# Get the average of each column within the GroupBy object
# age_group[["Age"]].mean()

age_group_test = age_group.count()
age_group_test

# delete the extra columns (NOT NEEDED)
del age_group_test['Price']
del age_group_test['SN']
del age_group_test['Item ID']
del age_group_test['Item Name']
del age_group_test['Purchase ID']
del age_group_test['Gender']

# Convert 'Age' into 'Total Count'
age_group_test = age_group_test.rename(columns={"Age": "Total Count"})

# to find the amount of genders 
age_total = age_group_test.sum()

# outcome we get is 'Total Count'        : '576'
#                   'Percent of Players' : '0'
#                    We want '576'
age_total = age_total["Total Count"]

# the amount of players with certain age... found in 'bins' of 'age_group_test' data frame... and divide it by the amount of players
less_than_ten_amount = age_group_test["Total Count"]["<10"] 
ten_to_fourteen_amount = age_group_test["Total Count"]["10-14"] 
fifteen_to_ninteen_amount = age_group_test["Total Count"]["15-19"] 
twenty_to_twentyfour_amount = age_group_test["Total Count"]["20-24"] 
twentyfive_to_twentynine_amount = age_group_test["Total Count"]["25-29"] 
thirty_to_thirtyfour_amount = age_group_test["Total Count"]["30-34"] 
thirtyfive_to_thirtynine_amount = age_group_test["Total Count"]["35-39"] 
fortyplus_amount = age_group_test["Total Count"]["40+"] 

# Add Column 'Percentage of Players' with its 'values'
age_group_test.loc[:, 'PercentageofPlayers'] = [less_than_ten_amount/age_total, 
                                                  ten_to_fourteen_amount/age_total,
                                                  fifteen_to_ninteen_amount/age_total,
                                                  twenty_to_twentyfour_amount/age_total,
                                                  twentyfive_to_twentynine_amount/age_total,
                                                  thirty_to_thirtyfour_amount/age_total,
                                                  thirtyfive_to_thirtynine_amount/age_total,
                                                  fortyplus_amount/age_total]

# convert the 'PercentageofPlayers' values into percetange and string ( example:  .8346121 to 83.46121  )
age_group_test.PercentageofPlayers = (age_group_test.PercentageofPlayers * 100).astype(str)

#TO ROUND VALUES UP AND ADD '%'
age_group_test["Percentage of Players"] = age_group_test["PercentageofPlayers"].astype(float).map("{:,.2f}%".format)

# DEL the extra COLUMN
del age_group_test['PercentageofPlayers']

age_group_test.head(8)


,Total Count,Percentage of Players
,,
<10,17,2.95%
10-14,22,3.82%
15-19,107,18.58%
20-24,258,44.79%
25-29,77,13.37%
30-34,52,9.03%
35-39,31,5.38%
40+,12,2.08%
